<a href="https://colab.research.google.com/github/np1011/StaffingandGraduationRatesPhiladelphiaSchools/blob/main/DSCI_632_Final(1).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# DSCI 632 Final

Your final is to use the dataset of your choice and the techniques in this class to do the following:

1. Define and describe the dataset (10 points)
1. Clean the data (10 points)
1. Transform the clean data (10 points)
1. Show your analysis of the data (10 points)

## Ground Rules

1. Explain everything you're doing with pyspark in the code cells using [markdown](https://www.markdownguide.org/cheat-sheet) in text cells. Help the reader understand why you're doing each step so they can re-create it. Remember, this is an assignment to show how you analyze data to a potential employer. Don't use code comments to explain things.
1. All of the sections are heading 1 in markdown, so use heading 2 to write your explainations, and heading 3 for any sub-headers. Check in your table of contents view in colab to make sure each point is listed before you turn this in.
1. Don't hesisate to use multiple code/text cells in each section, as long as they're all labeled and described.
1. Assume that the reader doesn't have access to the dataset on your local machine. Provide a link to the dataset you're using, or if able, include code to copy it from a public source. Don't rely on uploading from your local machine. (Importing from Google Drive is ok, as long as you provide the link to the data)
1. If using GCP services, include screenshots from your console if there's a step that you aren't able to re-produce in code. Using the SDK is always preferred, but you won't lose points for using screenshots and explaining them.
1. Import the data ONCE, then transform it to fit your analysis.
1. Don't overwrite data, make new columns for new transformations. You can always drop columns later, but you can't get overwritten values back.
1. Feel free to work with classmates, but all work submitted must be your own.
1. Make sure to run disconnect the runtime and re-run the notebook at least once before turning in. If you are getting certain Java runtime errors, this might also help.

## Extra Credit Opportunities!

- If you provide a link to this notebook on your public GitHub page instead of turning it in as an attachment, you will get 2 points extra credit.
- Extra credit will be given for using the [Google Cloud SDK](https://cloud.google.com/sdk/) to create/use/destroy any cloud resources, up to 2 points per section, 8 points total.
 - Don't be afraid to look at the GCP example colab notebooks from the class notes, the GCP documentation, or GitHub for examples.
- If this notebook is self-contained, you will get 5 points extra credit. (Requirements below)
 - The data is imported without relying on Google Drive, as the paths to data in Google Drive are user-specific. Downloads from public storage buckets/services are fine. (1 point)
 - It be run from start to finish without making any changes to paths, code, or variable names. (1 point)
 - All calls to external services, including GCP, are done programatically, no screenshots explaining how it worked in a browser console. (Using an API or SDK, 2 points)
 - All cloud resources are destroyed at the end of the notebook in a seperate section (Also via API or SDK, 1 point)

---

## Section 1
### Define and describe the dataset

10 points

Import the dataset, and describe why you'll be analyzing in it. You can summarize a few columns, show a more information on the relavent features, or but help the reader understand what the dataset is, what is in it, and why you picked it.

Some questions that it might help you to answer:
1. Why are you choosing this dataset?
1. What variables will you use?
1. What analysis(es) will you run?
1. Do you have any hypotheses? What are they?

### This section should include one or more of the following:
- A histogram of several features relavent to your analysis
- The schema of the dataset, with the datatypes assigned correctly
- A text cell with explainations of the relavent features in [markdown](https://www.markdownguide.org/cheat-sheet)
- Use Spark SQL or built-in methods to show a range of values
- Most common words/n-grams found

## Section 1: Part 0
## Overview

1. Why are you choosing this dataset?

I selected this dataset in large part because I happen to know a lot of public school teachers who, shortly before this assignment, were discussing the challenges of working in schools with indadequate  staff (for example, not enough social workers). I wanted to investigate whether I could find data-based evidence on the relationship between school performance and staffing.

The Philadelphia School District (PSD) provides a wide array of metrics for looking into this question. I chose to look at Graduation Rates as the metric of success, since High School Graduation status is an important metric of future earning potential.

2. What variables will you use?

I plan to use some calculated variables - total staff to student ratio, for example, as well as the data points provided by the PSD. They have metrics such as - enrollment type (cityside vs neighborhood), staff category, whether or not students are considered disadvantaged. These can help illimunate some of the statistics, because while a school may have a lower overall graduation rate they may be relatively higher performing for disadvanted students, for example.

3. What analysis(es) will you run?

The primary analysis I would like to run is a Pearson and Spearman correlation analysis. This can answer my initial question of whether more staff correlates with higher graduation rates. I am also hoping it reveals other interesting patterns that I can dive into, indicating which variables correlate strongly with high and low gradation rates.


4. Do you have any hypotheses? What are they?

I believe there will be a relationship between staffing levels and graduation rates. I am curious, however, since I am looking at a single district with a shared budget (rather than districts relying on different tax bases), how that will play out. Will staff resources  be higher at low performing schools because of high-need student populations? Or do higher performing schools attract resources, with more staff supporting the success of schools with better graduation rates?


## Section 1: Part 1
## Environment Set Up

In Section: Part 1 I will setup the PySpark environment required to run the analysis.

In [ ]:
# Install Spark 3.0.1
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://archive.apache.org/dist/spark/spark-3.0.1/spark-3.0.1-bin-hadoop2.7.tgz
!tar xf spark-3.0.1-bin-hadoop2.7.tgz

In [ ]:
# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.0.1-bin-hadoop2.7"

In [ ]:
!python -m pip install --upgrade pyspark==2.4.0
!python -m pip install -q findspark

In [ ]:
import findspark
findspark.init()

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
from pyspark.sql import SparkSession

In [ ]:
from pyspark import SparkContext

In [ ]:
APP_NAME = "PHLSchools"

In [ ]:
spark = SparkSession.builder.appName(APP_NAME).getOrCreate()

In [ ]:
spark

In [ ]:
from pyspark.sql.functions import col
from pyspark.sql.functions import countDistinct, split, monotonically_increasing_id, udf, collect_list

##Section 1 : Part 2
## Import data from GitHub Repository

The required data comes from 3 different locations on the  [Philadelphia School District (PSD) website](https://www.philasd.org/performance/programsservices/open-data/). In Section 1, I will load each of the 3 data sources into dataframes with the correct schemas, check that they've loaded properly and review important variables from each.

In [ ]:
#Load Data from Public GitHub Repository
!git clone https://github.com/np1011/StaffingandGraduationRatesPhiladelphiaSchools

!ls

### I. School List Data

**The important variables here will be**:
* School Name
* School Level (Elementary, Middle or High School)
* Admission Type (Whether a school population is based on zip code or a filtered admissions process)


First file is the School List for 2022-2023. I am only going to examine currently operating schools, so one year of current data is the appropriate reference point. This file will give a reference point for current schools and also provide the IDs used across datasets so they can be joined cleanly.

Source: https://www.philasd.org/performance/programsservices/open-data/school-information/#school_lists

In [ ]:
#Read in data file, infer schema and display first 5 rows
schoolList0 = spark.read.options(header = 'True', inferSchema='True',delimiter=',') \
  .csv("/content/StaffingandGraduationRatesPhiladelphiaSchools/2022_2023_Master_School_List.csv")

schoolList0.show(5)

#Check Schema
schoolList0.printSchema()

### Important Data Points from the School List

*   There are 56 Unique High Schools in the PSD
*   The below cell also displays school name with Admission Type, and count of High Schools by Admission Type since I believe Admission Type will be an important variable in graduation rate. Selecting the pool of students will likely improve graduation rates.



In [ ]:
#Get list of All PSD High Schools
schoolList0.select('Publication Name', 'Admission Type', 'ULCS Code').where(col('School Level') == 'High').show(60,truncate = False)
schoolList0.select('Publication Name').where(col('School Level') == 'High').count()
admissiontypecount = schoolList0.where(col('School Level') == 'High').groupBy('Admission Type').count()
admissiontypecount.show(truncate = False)

### II. Graduation Rate Data

The important variables here will be:
* cohort - year the class started 9th grade
* schoolname - Name of the school
* schoolid_ulcs - ID to use to normalize data with School List
* rate_type - 4, 5 or 6 year graduation rate for the cohort
* demon - total number of students in the cohort
* score - ratio of total graduates to denom
* group & subgroup - define aspects of a cohort (i.e. English Language Learner (ELL), Economically Disadvantaged)

Next I will upload a file of graduation rates. It goes from 2013-2022. Employee data is available starting in 2014, so this date range matches up well and I do not need to go back further.

Source: https://www.philasd.org/performance/programsservices/open-data/school-performance/#school_graduation_rates

In [ ]:
from pyspark.sql.types import StructType,StructField, StringType, IntegerType, FloatType

In [ ]:
#Read in data file, infer schema and display first 5 rows
customSchema = StructType([
    StructField('cohort', StringType(), True),
    StructField('schoolid_ulcs', IntegerType(), True),
    StructField('schoolname', StringType(), True),
    StructField('sector', StringType(), True),
    StructField('rate_type', StringType(), True),
    StructField('group', StringType(), True),
    StructField('subgroup', StringType(), True),
    StructField('demon', IntegerType(), True),
    StructField('num', IntegerType(), True),
    StructField('score', FloatType(), True),
])

gradRate0 = spark.read.format("csv").option("header", "true").option("delimiter", ",").schema(customSchema).load("/content/StaffingandGraduationRatesPhiladelphiaSchools/Graduation_Rates_2023-02-15.csv")

gradRate0.show(truncate = False)

#Check Schema
gradRate0.printSchema()

gradRate0.count()

Since null values cannot be used for analysis (if there is no graduation rate data there is nothing to compare). I am proactivey removing all the nulls at this step. I am doing this in Section 1 instead of Section 2 so I can create a clean Box Plot to view.

Before Dropping Nulls there were 26,163 rows. After Dropping Nulls there are 15,362

In [ ]:
#Comparing the Nulls to the School List, the null values are schools that aren't active in the current school list. Drop them
cleanGradRate = gradRate0.na.drop()
cleanGradRate.count()

### Find Outliers in Graduation Rates

There are some outliers with exceedingly low graduation rates however, reviewing them, I can see that when the dataframes are combined in Section 2 these should be remedied. These schools are not included in the School List data, which is likely a reason for their outlier status (data may be incomplete or schools are no longer operating).

Additionally it is important to note that the 'cohort' variable referes to the year a student started 9th grade, not their graduation year.

In [ ]:
#Select all student graduation rates
allStudentGrad = cleanGradRate.filter((col('group') == 'All Students') & (col('cohort') == '2018-2019') & (col('rate_type') == '4-Year Graduation Rate')).select(col('score'), col('schoolname'))

#Show Ordered Graduation Rates
allStudentGradDisplay = allStudentGrad.orderBy('score').show(100, truncate = False)


### Visualize Graduation Rate Distribution with Box Plot

Use the allStudentGrad dataframe, that selects the 4-year graduation rate for the most recent available cohort (2018-2019). Convert the dataframe to pandas, then display a box plot to visualize the mean and outliers.

You can see on this plot that on average, PSD schools are graduated less than 80% of this cohort of students in 4-years.

In [ ]:
#Display Box Plot of Graduation Rates
pandaGrad = allStudentGrad.toPandas()

pandaGrad['score'].plot.box()

### III. Employee Data

Important Variables:
* PAY_RATE_TYPE: Hourly or Salaried
* PAY_RATE
* TITLE_DESCRIPTION : Job Title
* HOME_ORGANIZATION : School ID that is normalized across dataframes (School names are formatted differently than in graduation rate data, so cannot be used to compare)
* RUN_DATE: Date data was pulled, allows for matching data to school year

The final piece of data is employee information. This is not as neat of a dataset because data is provided in CSV's year by year. I will upload the files, then combine them into a single dataframe.

This data had to be cleaned up before I could even load it. The files were not named consistently (different capitalization and underscore usage) and some zip files contained Read Me while others did not. The data is also provided multiple times per year, creating duplicates, and not always for the same month. To avoid inconsistency we will look at employment in July of each year - a snapshot of who was employed just before the school year started. 2021-2022 also only has July data so this allows for consistency.

In Section 2, this data will be consolidated into statistics (i.e. average employee pay, average count of employee per student)

In [ ]:
#https://stackoverflow.com/questions/60261615/how-can-i-merge-these-many-csv-files-around-130-000-using-pyspark-into-one-lar
rdd = spark.sparkContext.emptyRDD()

schema = StructType([
  StructField('LAST_NAME', StringType(), True),
  StructField('FIRST_NAME', StringType(), True),
  StructField('PAY_RATE_TYPE', StringType(), True),
  StructField('PAY_RATE', IntegerType(), True),
  StructField('TITLE_DESCRIPTION', StringType(), True),
  StructField('HOME_ORGANIZATION', StringType(), True),
  StructField('HOME_ORGANIZATION_DESCRIPTION', StringType(), True),
  StructField('ORGANIZATION_LEVEL', StringType(), True),
  StructField('TYPE_OF_REPRESENTATION', StringType(), True),
  StructField('GENDER', StringType(), True),
  StructField('RUN_DATE', StringType(), True),
  ])
result = spark.createDataFrame(rdd, schema)

for year in range(2014,2022):
    year_str = str(year)
    year_next = str(year+1)

    df = spark.read.format("csv").option("header", "true").option("delimiter", ",").schema(schema).load("/content/StaffingandGraduationRatesPhiladelphiaSchools/Employee_Data_All_Years_2023/Employee_Data_" + year_str + "_" + year_next + "/employee_information_July"+year_str+".csv")

    if result:
        result = result.union(df)
    else:
        result = df
result.show(10)
result.count()
result.printSchema()

### Checking that Employee Data was Loaded Successfully

Below I am reviewing the total run dates to make sure all the employee data files that I intended to upload were in fact included in the upload.

In [ ]:
#Checking to make sure all the files were included (they each have a unique run date)
result.select("RUN_DATE").distinct().show()

#Get count of total rows
print('Total Rows of Employee Data')
result.count()

# Section 2: Clean the data

10 points

Not every dataset is going to be ready to use right away. Take steps to fix incorrect inputs, remove null values, and assign datatypes that fit each feature.

### This section should include one or more of the following:
- A count of rows before and after dropping data with null or incorrect values, including an explaination of why removing the rows was necessary in [markdown](https://www.markdownguide.org/cheat-sheet).
- Renaming a column, adding a new one, or casting an existing one as a different datatype.
- Remove punctuation marks, symbols, etc from the data, and convert all upper-case lettering to lower-case.
- Remove or fix rows that have bad values/data.

## Clean Up & Combine School List Data with Graduation Rate Data

First, in order to include only the currently active PSD high schools, I am going to filter the Graudation Rate Data by the School List data based on the ULCS code.

First, normalize the ID column names in gradRate and schoolList to 'ULCS'. Then limit the schoolList0 dataframe to High Schools only.

In [ ]:
#Normalize the column names
cleanGradRate = cleanGradRate.withColumnRenamed("schoolid_ulcs","ULCS")
schoolListClean = schoolList0.withColumnRenamed("ULCS Code", "ULCS")

#Limit the School List to Only High Schools
highSchoolList = schoolListClean.filter(col("School Level") == "High")

Perform a join on ULCS ID. This will limit the grad rate list to only the currently active high schools.

There were 56 total high schools, but only 46 remain in this join. That is because schools some schools were not included in the graduation data and some schools provided null values and were dropped in Section 1.

The following edits will be made:

* Drop School List columns that will not be used because it's an extensive number of columns and many columns just have the same value in all rows.

* Add columns named Freshman and Senior, based on the cohort column, to indicate the year the cohort started high school and the year they would graduate, if graduating in 4 years. This can be used to compare to employment data by year.

* Filter to 4-Year-Graduation-Rate only so that there is a single metric to look to

In [ ]:
#Join School List and Grad Data by ULCS ID
gradFiltered = cleanGradRate.join(highSchoolList, ["ULCS"], "inner")

#Check new unique school count
gradFilteredCount = gradFiltered.select(countDistinct("ULCS"))
gradFilteredCount.show()

#Remove unnecessary columns
gradFiltered = gradFiltered.drop("Community School Cohort","CTE Status","City Council District","Federal Accountability Designation","Phasing-In","Phasing-Out","Title I Designation","FACE Liaison Name","FACE Liaison Email","FACE Liaison Phone Number","PA Code","NSC Code","NCES Code", "AUN Code", "SRC School ID", "Abbreviated Name","Grade Span at Scale","GPS Location","Multiple Addresses","Street Address", "City", "State", "Zip Code", "Phone Number", "Fax Number", "Website", "School Leader Name", "Learning Network", "Assistant Superintendent", "Grade K", "Grade 1", "Grade 2", "Grade 3", "Grade 4", "Grade 5", "Grade 6", "Grade 7", "Grade 8", "Grade 9", "Grade 10", "Grade 11", "Grade 12")

#Add Freshman & Senior Columns, Cast as Integers
gradFiltered1 = gradFiltered.withColumn("freshman", split(gradFiltered["cohort"], "-").getItem(0))
gradFiltered1 = gradFiltered1.withColumn("senior",col("freshman") + 4)
gradFiltered1 = gradFiltered1.withColumn("senior",col("senior").cast("Integer"))
gradFiltered1 = gradFiltered1.withColumn("freshman",col("freshman").cast("Integer"))
gradFiltered2 = gradFiltered1.where(col("rate_type") == "4-Year Graduation Rate")
gradFiltered2.show()

## Clean up Employment Data

I am going to perform minimal clean up of the employment data in this section, because this data will primarily be transformed in Section 3 into calculated data columns.

* Rename HOME_ORGANIZATION to ULCS, for consistency
* Add a new column "School_Year_Start" based on the Run Date, to indicate the fall of the school year referenced by the employee data.
* Add Employee ID column, so unique employees can be easily counted
* There are a lot of employees who are not tied to a particular school, from bus drivers to school district accountants, as well as people who work at Middle & Elementary Schools. I will remove them, so we can look at only employees assigned to the 46 active schools for which we have graduation data.

All 46 Schools are correcty represented in the Employee Data. The total employee count, filtered for only employees assigned to these schools is 26,819 down from 150,895 in the original dataset (as calculated in Section 1). This is for 9 years of data. So on average there are 2979 employees per year, with 64 employees per school.

In [ ]:
#Rename Home_Organization to ULCS
employeeData = result.withColumnRenamed("HOME_ORGANIZATION","ULCS")

#Create School Year Start column
employeeData = employeeData.withColumn("School_Year_Start", split(employeeData["RUN_DATE"], "/").getItem(2))

#Get List of 46 ULCS Codes that are being examined
ULCSGroup = gradFiltered.select("ULCS").distinct()
ULCSList = ULCSGroup.select("ULCS").rdd.flatMap(lambda x: x).collect()

#Filter Dataframe to include only 46 ULCS Codes
employeeData1 = employeeData.filter(employeeData['ULCS'].isin(ULCSList))

In [ ]:
#Check to make sure all schools are included
employeeData1Count = employeeData1.select(countDistinct("ULCS"))
employeeData1Count.show()

#Get Count of Employees for these 46 schools only
print(employeeData1.count())

#Create Employee ID Column
employeeDataID = employeeData1.withColumn("Employee_ID", monotonically_increasing_id())

employeeDataID.show()

This is a quick check of the employees that have been dropped because they are not assigned to a ULCS high school. They are either middle/elementary school employees, administrators or support workers that do not match to a single school (i.e. Food Services Admin & Support)

In [ ]:
notSchool = employeeData.filter(~employeeData['ULCS'].isin(ULCSList))
notSchool.groupBy("TITLE_DESCRIPTION", "HOME_ORGANIZATION_DESCRIPTION", "ULCS").agg(countDistinct("Employee_ID")).show(truncate = False)

# Section 3: Transform the clean data

10 points

Once you have clean data, start to prepare it to fit your analysis tools. This might mean using custom code to normalize certain values, joining supplemental datasets, and/or preparing it for machine learning.

### This section should include one or more of the following:
- Write a UDF to perform a function, then use it to add a new column to your data. Explain why in [markdown](https://www.markdownguide.org/cheat-sheet)
- Join an outside data source. (It can be one you've prepared alongside the primary source you're using, as long as you link it)
- Split the data into train/test sets
- Create vectors for relavent features
- One-hot encode catagorical variables

### Calculate Data from Employee Data
* Employees per school per school year start
* Average salary (hourly & salaried) per school per school year start
* Teacher, Full-Time per school per year start
* Non-teachers per school per year start


In [ ]:
employeeYear = employeeData1.groupBy('School_Year_Start').agg(collect_list('Employee_ID').alias('Employee_Count')).show()

# Section 4: Show your analysis of the data

10 points

This is where the science happens. Use your data to show some kind of insight, and how you got there. Make the reader understand why it's important, and how they can get the same conslusion, and/or what would need to change to reach a different one.

### This section should include one or more of the following:
- Fit the data to a model
- Show the outcome of clustering, regression, and/or classification algorithms.
 - We used several in class, but you can use whatever fits your needs for this assignment
- Reccomend a product/item
- Use a SQL query to filter results
